In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer


In [2]:
df = pd.read_csv("POST-ARRIVAL_STUDENT_MENTAL_HEALTH_SURVEY_EXTENDED.csv")

print(df.shape)
df.head()


(148, 31)


,Timestamp,1. Age Group,2. Gender,3.Academic Level,4. Country you are currently studying in.,5. How often do you feel overwhelmed by your responsibilities abroad?,6.How often do you feel nervous or on edge since arriving abroad?,7. How difficult is it for you to relax after your daily activities while living abroad?,8. How often do you feel low energy or lack motivation due to stress?,9. How often do you struggle with concentration because of mental pressure?,...,21. How stressed are you about managing living expenses abroad?,22. Do financial difficulties affect your emotional well-being?,23. How anxious are you about long-term visa or immigration matters?,24. How confident are you in finding stable work or financial support abroad?,25. Do you feel emotionally supported by your family while living abroad?,26. How often do you feel homesick?,27. How comfortable are you sharing emotional struggles with family or friends?,28. Do you get sufficient sleep (7–8 hours)?,29. How often do you engage in physical activity or exercise?,"30. Do you practice relaxation activities (prayer, meditation, breathing exercises)?"
0,2025/12/08 11:15:55 PM GMT+6,22-26,Male,Masters,USA,2.0,4.0,Yes,4.0,4.0,...,4.0,Yes,5.0,2.0,Yes,2.0,4.0,No,1.0,No
1,2025/12/15 10:01:13 PM GMT+6,18-22,Male,Masters,Australia,1.0,2.0,1,1.0,2.0,...,3.0,Yes,2.0,4.0,Yes,4.0,4.0,No,4.0,Yes
2,2025/12/15 10:03:03 PM GMT+6,26-30,Female,Masters,usa,4.0,4.0,3,4.0,4.0,...,4.0,Yes,4.0,2.0,No,4.0,4.0,No,4.0,No
3,2025/12/15 10:06:44 PM GMT+6,26-30,Female,Masters,Australia,4.0,4.0,4,3.0,4.0,...,4.0,Yes,4.0,3.0,Yes,4.0,4.0,No,4.0,Yes
4,2025/12/15 10:06:50 PM GMT+6,18-22,Male,Masters,Sweeden,2.0,2.0,2,2.0,4.0,...,3.0,Yes,3.0,4.0,Yes,3.0,4.0,Yes,4.0,Yes


In [3]:
df = df.drop(df.index[149:175])


In [4]:
depression_cols = [
    "5. How often do you feel overwhelmed by academic workload after arrival?",
    "6. How often do you feel nervous or anxious in the new environment?",
    "8. How often do you worry about coping with academic pressure?",
    "9. How often do you feel low energy or lack motivation after arrival?",
    "11. How anxious are you about financial or living expenses?",
    "12. How often do you overthink problems related to studies or生活 abroad?",
    "14. How stressed do you feel about cultural adjustment?",
    "20. How often do you feel pressure to perform well academically?",
    "22. How concerned are you about social isolation after arrival?",
    "24. Do you get sufficient sleep (7_8_ hours) after arrival?",
    "30. How stressed are you about managing studies and personal life?"
]


In [5]:
# -------------------------------
# AUTO-DETECT + FIX DEPRESSION COLUMNS (POST-ARRIVAL)
# -------------------------------

# keywords that identify depression-related questions
keywords = [
    "overwhelmed",
    "nervous",
    "anxious",
    "worry",
    "low energy",
    "motivation",
    "stressed",
    "pressure",
    "isolated",
    "sleep"
]

# auto-select matching columns
depression_cols = [
    col for col in df.columns
    if any(k.lower() in col.lower() for k in keywords)
]

print("Detected Depression Columns:")
for c in depression_cols:
    print(c)

# convert to numeric
df[depression_cols] = df[depression_cols].apply(
    pd.to_numeric, errors="coerce"
)

# reverse-code sleep (if present)
sleep_cols = [c for c in depression_cols if "sleep" in c.lower()]
for c in sleep_cols:
    df[c] = 6 - df[c]

# fill missing values
for col in depression_cols:
    df[col].fillna(df[col].median(), inplace=True)

# create depression score
df["Depression_Score"] = df[depression_cols].mean(axis=1)

# create multiclass label
def depression_level(score):
    if score <= 2.0:
        return "No/Minimal"
    elif score <= 3.0:
        return "Mild"
    elif score <= 4.0:
        return "Moderate"
    else:
        return "Severe"

df["Depression_Level"] = df["Depression_Score"].apply(depression_level)

print("\nDepression Level Distribution:")
print(df["Depression_Level"].value_counts())



Detected Depression Columns:
  5. How often do you feel overwhelmed by your responsibilities abroad?  
6.How often do you feel nervous or on edge since arriving abroad?  
  8. How often do you feel low energy or lack motivation due to stress?   
 9. How often do you struggle with concentration because of mental pressure?  
 12. How often do you feel socially isolated or lonely abroad?  
  17. Do you feel pressure to achieve high academic results?  
  19. How often do you feel stressed before exams or deadlines?   
  21. How stressed are you about managing living expenses abroad?  
  23. How anxious are you about long-term visa or immigration matters?  
  28. Do you get sufficient sleep (7–8 hours)?  

Depression Level Distribution:
Depression_Level
Moderate      92
No/Minimal    31
Mild          25
Name: count, dtype: int64


C:\Users\Asus\AppData\Local\Temp\ipykernel_10864\3169787499.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)


In [6]:
X = df.drop(columns=["Depression_Level", "Depression_Score"])
y = df["Depression_Level"]


In [7]:
# Drop target and helper column
X = df.drop(columns=["Depression_Level", "Depression_Score"])
y = df["Depression_Level"]

print(X.shape, y.shape)


(148, 31) (148,)


In [8]:
categorical_cols = X.select_dtypes(include=["object"]).columns
numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns

print("Categorical features:", len(categorical_cols))
print("Numeric features:", len(numeric_cols))


Categorical features: 10
Numeric features: 21


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [10]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, categorical_cols)
    ]
)


In [11]:
from sklearn.linear_model import LogisticRegression

In [12]:
logreg_base = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(
        multi_class="multinomial",
        solver="lbfgs",
        max_iter=1000,
        class_weight="balanced",
        random_state=42
    ))
])


In [13]:
logreg_base.fit(X_train, y_train)


c:\Users\Asus\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['  17. Do you feel pressure to achieve high academic results?  '
 '  19. How often do you feel stressed before exams or deadlines?   '
 '  28. Do you get sufficient sleep (7–8 hours)?  ']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\Asus\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['  5. How often do you feel overwhelmed by your responsibilities abroad?  ',
       '6.How often do you feel nervous or on edge since arriving abroad?  ',
       '  8. How often do you feel low energy or lack motivation due to stress?   ',
       ' 9....
       ' 14. Do you feel accepted by classmates, peers, or community members?  ',
       '  22. Do financial difficulties affect your emotional well-being?  ',
       '  25. Do you feel emotionally supported by your family while living abroad?  ',
       '  30. Do you practice relaxation activities (prayer, meditation, breathing exercises)?  '],
      dtype='object'))])),
                ('classifier',
                 LogisticRegression(class_weight='balanced', max_iter=1000,
                                    multi_class='multinomial',
                                    random_state=42))])

In [14]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred_base = logreg_base.predict(X_test)

print("Logistic Regression Baseline\n")
print(classification_report(y_test, y_pred_base))

print("Confusion Matrix:\n")
print(confusion_matrix(y_test, y_pred_base))


Logistic Regression Baseline

              precision    recall  f1-score   support

        Mild       1.00      0.80      0.89         5
    Moderate       1.00      1.00      1.00        19
  No/Minimal       0.86      1.00      0.92         6

    accuracy                           0.97        30
   macro avg       0.95      0.93      0.94        30
weighted avg       0.97      0.97      0.97        30

Confusion Matrix:

[[ 4  0  1]
 [ 0 19  0]
 [ 0  0  6]]


c:\Users\Asus\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['  17. Do you feel pressure to achieve high academic results?  '
 '  19. How often do you feel stressed before exams or deadlines?   '
 '  28. Do you get sufficient sleep (7–8 hours)?  ']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


In [15]:
param_grid = {
    "classifier__C": [0.01, 0.1, 1, 10],
    "classifier__penalty": ["l2"],
    "classifier__solver": ["lbfgs", "newton-cg", "sag"]
}


In [16]:
from sklearn.model_selection import GridSearchCV

logreg_tuned = GridSearchCV(
    estimator=logreg_base,
    param_grid=param_grid,
    scoring="f1_macro",
    cv=5,
    n_jobs=-1,
    verbose=1
)


In [17]:
logreg_tuned.fit(X_train, y_train)


Fitting 5 folds for each of 12 candidates, totalling 60 fits


c:\Users\Asus\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['  17. Do you feel pressure to achieve high academic results?  '
 '  19. How often do you feel stressed before exams or deadlines?   '
 '  28. Do you get sufficient sleep (7–8 hours)?  ']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\Asus\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['  5. How often do you feel overwhelmed by your responsibilities abroad?  ',
       '6.How often do you feel nervous or on edge since arriving abroad?  ',
       '  8. How often do you feel low energy or lac...
       '  30. Do you practice relaxation activities (prayer, meditation, breathing exercises)?  '],
      dtype='object'))])),
                                       ('classifier',
                                        LogisticRegression(class_weight='balanced',
                                                           max_iter=1000,
                                                           multi_class='multinomial',
                                                           random_state=42))]),
             n_jobs=-1,
             param_grid={'classifier__C': [0.01, 0.1, 1, 10],
                         'classifier__penalty': ['l2'],
                         'classifier__solver': ['lbfgs', 'newton-cg', 'sag']},
             scoring='f1_macro', verbose=1)

In [18]:
print("Best Parameters:")
print(logreg_tuned.best_params_)


Best Parameters:
{'classifier__C': 0.01, 'classifier__penalty': 'l2', 'classifier__solver': 'lbfgs'}


In [19]:
y_pred_tuned = logreg_tuned.predict(X_test)

print("Logistic Regression Tuned\n")
print(classification_report(y_test, y_pred_tuned))

print("Confusion Matrix:\n")
print(confusion_matrix(y_test, y_pred_tuned))


Logistic Regression Tuned

              precision    recall  f1-score   support

        Mild       1.00      0.80      0.89         5
    Moderate       1.00      1.00      1.00        19
  No/Minimal       0.86      1.00      0.92         6

    accuracy                           0.97        30
   macro avg       0.95      0.93      0.94        30
weighted avg       0.97      0.97      0.97        30

Confusion Matrix:

[[ 4  0  1]
 [ 0 19  0]
 [ 0  0  6]]


c:\Users\Asus\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['  17. Do you feel pressure to achieve high academic results?  '
 '  19. How often do you feel stressed before exams or deadlines?   '
 '  28. Do you get sufficient sleep (7–8 hours)?  ']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


In [20]:
feature_names = (
    logreg_tuned.best_estimator_
    .named_steps["preprocessor"]
    .get_feature_names_out()
)

coef = logreg_tuned.best_estimator_.named_steps["classifier"].coef_

coef_df = pd.DataFrame(coef, columns=feature_names)

coef_df.abs().mean().sort_values(ascending=False).head(15)


num__ 9. How often do you struggle with concentration because of mental pressure?                      0.121173
num__  21. How stressed are you about managing living expenses abroad?                                 0.094873
num__16. How stressful is the academic workload in your institution?                                   0.089466
num__6.How often do you feel nervous or on edge since arriving abroad?                                 0.085176
num__24. How confident are you in finding stable work or financial support abroad?                     0.079553
num__  8. How often do you feel low energy or lack motivation due to stress?                           0.074431
num__10. How confident are you in handling emotional ups and downs abroad?                             0.070866
cat__Timestamp_2025/12/15 10:06:50 PM GMT+6                                                            0.070470
num__ 18. How challenging is it to balance academics with part-time work (if applicable)?              0

In [21]:
coef_df.abs().mean().sort_values(ascending=False).head(15)


num__ 9. How often do you struggle with concentration because of mental pressure?                      0.121173
num__  21. How stressed are you about managing living expenses abroad?                                 0.094873
num__16. How stressful is the academic workload in your institution?                                   0.089466
num__6.How often do you feel nervous or on edge since arriving abroad?                                 0.085176
num__24. How confident are you in finding stable work or financial support abroad?                     0.079553
num__  8. How often do you feel low energy or lack motivation due to stress?                           0.074431
num__10. How confident are you in handling emotional ups and downs abroad?                             0.070866
cat__Timestamp_2025/12/15 10:06:50 PM GMT+6                                                            0.070470
num__ 18. How challenging is it to balance academics with part-time work (if applicable)?              0

In [22]:
class_labels = logreg_tuned.best_estimator_.named_steps["classifier"].classes_

for i, cls in enumerate(class_labels):
    print(f"\nTop features for class: {cls}")
    (
        coef_df.iloc[i]
        .abs()
        .sort_values(ascending=False)
        .head(10)
    )



Top features for class: Mild

Top features for class: Moderate

Top features for class: No/Minimal


In [23]:
top_features = coef_df.abs().mean().sort_values(ascending=False).head(10).index

coef_df[top_features]


,num__ 9. How often do you struggle with concentration because of mental pressure?,num__ 21. How stressed are you about managing living expenses abroad?,num__16. How stressful is the academic workload in your institution?,num__6.How often do you feel nervous or on edge since arriving abroad?,num__24. How confident are you in finding stable work or financial support abroad?,num__ 8. How often do you feel low energy or lack motivation due to stress?,num__10. How confident are you in handling emotional ups and downs abroad?,cat__Timestamp_2025/12/15 10:06:50 PM GMT+6,num__ 18. How challenging is it to balance academics with part-time work (if applicable)?,num__ 11. How difficult was cultural adjustment after arriving?
0,0.122714,-0.129166,-0.079664,-0.054302,0.086434,-0.017902,-0.082595,0.105705,-0.002068,0.102212
1,0.059046,0.142310,0.134199,0.127764,-0.119329,0.111646,-0.023704,-0.047032,0.105240,-0.000956
2,-0.181760,-0.013144,-0.054535,-0.073462,0.032895,-0.093744,0.106299,-0.058673,-0.103173,-0.101255
